In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
dataset = 'Asset10k.xlsx'
df = pd.read_excel(dataset)

In [4]:
df

,AssetType,SerialNo,Manufacturer,ModelNbr,Refrigerant,RefrigerantQty
0,CHILLED BEAMS - PASSIVE,NaN,NaN,NaN,NaN,NaN
1,CHILLED BEAMS - PASSIVE,NaN,NaN,NaN,NaN,NaN
2,CHILLED BEAMS - PASSIVE,NaN,NaN,NaN,NaN,NaN
3,CHILLED BEAMS - PASSIVE,0,NaN,O,NaN,NaN
4,CHILLED BEAMS - PASSIVE,NaN,NaN,FRENGER,NaN,NaN
...,...,...,...,...,...,...
10148,CASSETTE SPLIT SYSTEM,329,MODULEX,PUH-4YKSA,NaN,0.0
10149,CASSETTE SPLIT SYSTEM,NaN,MODULEX,PUH-3YKA,NaN,0.0
10150,WALL MOUNTED SPLIT SYSTEM,H803 03649 NF,MITSUBISHI,FDC306HEN3,NaN,0.0
10151,WALL MOUNTED SPLIT SYSTEM,833325020,MITSUBISHI,SRC35ZG-S,NaN,0.0


In [29]:
X = df.iloc[:, 1:]
y = df['AssetType']

In [30]:
X

,SerialNo,Manufacturer,ModelNbr,Refrigerant,RefrigerantQty
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,0,NaN,O,NaN,NaN
4,NaN,NaN,FRENGER,NaN,NaN
...,...,...,...,...,...
10148,329,MODULEX,PUH-4YKSA,NaN,0.0
10149,NaN,MODULEX,PUH-3YKA,NaN,0.0
10150,H803 03649 NF,MITSUBISHI,FDC306HEN3,NaN,0.0
10151,833325020,MITSUBISHI,SRC35ZG-S,NaN,0.0


In [31]:
y.head()

0    CHILLED BEAMS - PASSIVE
1    CHILLED BEAMS - PASSIVE
2    CHILLED BEAMS - PASSIVE
3    CHILLED BEAMS - PASSIVE
4    CHILLED BEAMS - PASSIVE
Name: AssetType, dtype: object

In [32]:
special_values = ['NA', 'NULL', 'UNKNOWN', 'UNABLE TO LOCATE', 'NO ID', 'NO ACCESS', 'UNKOWN', 'NaN', 'na']

for column in X.columns:
    X[column] = X[column].replace(special_values, pd.NA)

In [33]:
# Label encoding for both X and y
label_encoder = LabelEncoder()

for column in X.select_dtypes(include=['object']).columns:
    X[column] = label_encoder.fit_transform(X[column].astype(str))

y = label_encoder.fit_transform(y.astype(str))

In [34]:
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

C:\ML\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)


In [35]:
# Split the dataset into training and testing sets
X_train_sampled, X_test, y_train_sampled, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

In [36]:
# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

In [37]:
# Train the classifier on the training data
rf_classifier.fit(X_train_sampled, y_train_sampled)

RandomForestClassifier(random_state=42)

In [38]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

In [39]:
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

C:\ML\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ML\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ML\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ML\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defi

In [41]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.6518956179222059


In [17]:
y_pred_actual = label_encoder.inverse_transform(y_pred)

In [18]:
y_pred_actual

array(['WALL MOUNTED SPLIT SYSTEM', 'GAS DUCT HEATER',
       'KITCHEN EXHAUST FAN', ..., 'AIR FAN - EXHAUST',
       'DUCTED SPLIT SYSTEM', 'DUCTED SPLIT SYSTEM'], dtype=object)

In [42]:
random_states = range(1,101)

In [44]:
for state in random_states:
    rf = RandomForestClassifier(n_estimators=100, random_state=state)
    rf.fit(X_train_sampled, y_train_sampled)
    y_pred = rf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Random State : {state} , Accuracy : {accuracy}")

Random State : 1 , Accuracy : 0.6474643032988675
Random State : 2 , Accuracy : 0.6474643032988675
Random State : 3 , Accuracy : 0.6479566715903495
Random State : 4 , Accuracy : 0.6464795667159035
Random State : 5 , Accuracy : 0.6484490398818316
Random State : 6 , Accuracy : 0.6484490398818316
Random State : 7 , Accuracy : 0.6484490398818316
Random State : 8 , Accuracy : 0.6484490398818316
Random State : 9 , Accuracy : 0.6474643032988675
Random State : 10 , Accuracy : 0.6484490398818316
Random State : 11 , Accuracy : 0.6479566715903495
Random State : 12 , Accuracy : 0.6469719350073855
Random State : 13 , Accuracy : 0.6435253569670113
Random State : 14 , Accuracy : 0.6484490398818316
Random State : 15 , Accuracy : 0.6474643032988675
Random State : 16 , Accuracy : 0.6514032496307238
Random State : 17 , Accuracy : 0.6440177252584933
Random State : 18 , Accuracy : 0.6459871984244214
Random State : 19 , Accuracy : 0.6474643032988675
Random State : 20 , Accuracy : 0.6450024618414574
Random St

In [45]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=63)

In [46]:
# Train the classifier on the training data
rf_classifier.fit(X_train_sampled, y_train_sampled)

RandomForestClassifier(random_state=63)

In [47]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

In [48]:
accuracy = accuracy_score(y_test, y_pred)

In [49]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.6528803545051699


In [50]:
import joblib

In [51]:
modelName = 'rf_63_AssetType'
joblib.dump(rf_classifier, modelName)

['rf_63_AssetType']

In [52]:
loaded_model = joblib.load('rf_63_AssetType')

In [54]:
new_prediction = loaded_model.predict(X_test)

In [55]:
new_prediction

array([117,  74,  88, ...,   6,  52,  52])

In [56]:
label_encoder_filename = 'label_encoder.joblib'

joblib.dump(label_encoder,label_encoder_filename)

['label_encoder.joblib']